**This is the final project in Data Analysis.**

Goals:
1. Analyze the cryptocurrency market in the selected time period
2. Try to predict the results of exchange rates (without focusing on events)

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import datetime as dt
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import plotly.express as px
from plotly import tools
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go

import xgboost as xgb

from sklearn.linear_model import LinearRegression as LinReg
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

Read the data and drop the symbols

In [2]:
df = pd.read_csv("crypto-markets.csv")
df = df.drop(['symbol'], axis=1)

In [3]:
groupByDate = df.groupby('date', as_index=False).sum()  # by date
groupByName = df.groupby('name', as_index=False).sum()  # by name
groupByMarketCap = df.groupby('market', as_index=False).sum()  # by market capitalization
groupByVolume = df.groupby('volume', as_index=False).sum()  # by market volume
groupByRanknow = df.groupby('ranknow', as_index=False).sum()  # by rank
groupBySpread = df.groupby('spread', as_index=False).sum()  # by spread (is the $USD difference between the high and low values for the day)

In [4]:
groupByRanknow.head(10)

,ranknow,open,high,low,close,volume,market,close_ratio,spread
0,1,4.666900e+06,4.807782e+06,4.507706e+06,4.670641e+06,2.961191e+12,7.729577e+13,1104.9198,300076.35
1,2,3.175345e+02,3.360558e+02,2.983503e+02,3.179459e+02,3.851715e+11,1.223008e+13,922.9499,37.15
2,3,2.584126e+05,2.685310e+05,2.466861e+05,2.584347e+05,1.034702e+12,2.517803e+13,626.1070,21844.86
3,4,1.140851e+02,1.208061e+02,1.069132e+02,1.142224e+02,4.663287e+10,2.045230e+12,757.8067,13.36
4,5,4.564125e+05,4.847501e+05,4.276237e+05,4.558391e+05,3.733380e+11,7.692101e+12,232.9049,57126.41
5,6,3.232716e+03,3.406773e+03,3.036923e+03,3.233056e+03,3.094760e+11,2.466003e+12,263.0976,369.89
6,7,6.354941e+04,6.640601e+04,6.035323e+04,6.355470e+04,2.825148e+11,3.336887e+12,1004.3938,6052.78
7,8,1.367704e+03,1.375792e+03,1.361541e+03,1.367835e+03,1.058689e+12,8.780396e+11,363.4101,14.13
8,9,2.028350e+03,2.433830e+03,1.639370e+03,2.033750e+03,2.895873e+09,6.984963e+09,9.8556,794.46
9,10,8.699238e+01,9.256046e+01,8.053808e+01,8.697284e+01,6.488140e+10,2.254232e+12,211.8733,11.84


Traders still like to analyze the concept of HLC (and OHLC|HL) [proof](https://www.mypivots.com/dictionary/definition/92/hlc-3)

In [5]:
df['hl_average'] = (df['high'] + df['low']) / 2
df['hlc_average'] = (df['high'] + df['low'] + df['close']) / 3
df['ohlc_average'] = (df['open'] + df['high'] + df['low'] + df['close']) / 4

Checking other currencies

In [6]:
top10 = df[(df['ranknow'] >= 1) & (df['ranknow'] <= 10)]
top10.name.unique()

array(['Bitcoin', 'XRP', 'Ethereum', 'Stellar', 'Bitcoin Cash', 'EOS',
       'Litecoin', 'Tether', 'Bitcoin SV', 'Cardano'], dtype=object)

*Volume* - All trades buys and sells that were made during that time (for example 24 hours like coinmarketcap does by default).

*Circulating supply* - number of coins mined and existing right now.

*Marketcap* = circulating supply multiply by price of coin.

In [7]:
fig = px.pie(top10, values='volume', names='name', title='Cryptocurrencies Top-10 by Transaction Volume')
fig.show()

In [8]:
fig = px.pie(top10, values='market', names='name', title='Cryptocurrencies Top-10 by Market capitalization')
fig.show()

In [9]:
fig = tools.make_subplots(subplot_titles=('Time'))
for name in top10.name.unique():
    currency = top10[top10['name'] == name]
    trace = go.Scatter(x=currency['date'], y=currency['ohlc_average'], name=name)
    fig.append_trace(trace, 1, 1)
    
fig['layout'].update(title='Top-10 Cryptocurrencies Comparison')
fig['layout']['yaxis1'].update(title='USD')
fig.show()

c:\python37\lib\site-packages\plotly\tools.py:465: DeprecationWarning:

plotly.tools.make_subplots is deprecated, please use plotly.subplots.make_subplots instead



Adding minor cryptocurrencies that not affect too much on the market

In [10]:
top10minorCurrencies = df[(df['ranknow'] >= 11) & (df['ranknow'] <= 21)]

top10minorCurrencies.name.unique()

array(['Monero', 'TRON', 'IOTA', 'Dash', 'NEM', 'Binance Coin', 'NEO',
       'Ethereum Classic', 'Zcash', 'Bitcoin Gold', 'Tezos'], dtype=object)

In [11]:
fig = px.pie(top10minorCurrencies, values='volume', names='name', title='Minor Cryptocurrencies by Transaction Volume')
fig.show()

In [12]:
fig = px.pie(top10minorCurrencies, values='market', names='name', title='Minor Cryptocurrencies by Market capitalization')
fig.show()

In [13]:
top10loserCoins = df[(df['ranknow'] >= max(df['ranknow']) - 10) & (df['ranknow'] <= max(df['ranknow']))]

top10loserCoins.name.unique()

array(['ALLCOIN', 'EmaratCoin', 'Agrolot', 'ZTCoin', 'PlayCoin [ERC20]',
       'Dragon Token', 'OBXcoin', 'APOT', 'Bgogo Token',
       'UniversalRoyalCoin', 'EtherDelta Token'], dtype=object)

In [14]:
fig = px.pie(top10loserCoins, values='volume', names='name', title='Loser Cryptocurrencies by Transaction Volume')
fig.show()

In [15]:
fig = px.pie(top10loserCoins, values='market', names='name', title='Loser Cryptocurrencies by Market capitalization')
fig.show()